<a href="https://colab.research.google.com/github/omkar109/Sleep-Disorder-ML-Model/blob/main/Sleep_Disorder_ML_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
import pandas as pd

#Hyperparameters
learning_rate = 0.01
batch_size = 12
epochs = 100
classification_threshold = 0.8
#If the model is more than 80% confident about a sleep disorder diagnosis, has
#sleep disorder will be marked as true

data = pd.read_csv("Sleep_health_and_lifestyle_dataset.csv") #imports dataset
data = data.sample(frac=1).reset_index(drop=True) #Shuffles the dataset

#Converts categorical features to one-hot encoded features
def one_hot_engineering(feature_list, data):
  for feature in feature_list:
    vocab = data[feature].unique().tolist()
    preprocessing_layer = tf.keras.layers.StringLookup(vocabulary=vocab)
    temp_feature = preprocessing_layer(tf.convert_to_tensor(data[feature])).numpy() -1.0
    prelayer_2 = tf.keras.layers.CategoryEncoding(num_tokens=len(vocab),output_mode="one_hot")
    one_hot_array = prelayer_2(tf.convert_to_tensor(temp_feature)).numpy()
    new_df = pd.DataFrame(one_hot_array, columns=[f'{category}' for category in vocab])
    data = pd.concat([data, new_df], axis=1)
  return data

#Creates a binary label and converts it into a floating point value
def label_engineering(data):
  data["Normal"] = data["Normal"] + data["Normal Weight"]
  data = data.drop("Normal Weight", axis=1)
  data["Has Sleep Disorder"] = (data["Sleep Disorder"] != "None").astype(float)
  return data

categorical_feature_list = ["Gender", "Occupation", "BMI Category"]
data = one_hot_engineering(categorical_feature_list, data)
data = label_engineering(data)


#Scale the features using normalization
def normalize_features(data):
  data_normalized = (data - data.mean()) / data.std()
  data_normalized = data_normalized.drop("Has Sleep Disorder", axis=1)
  data_normalized = pd.concat([data_normalized, data["Has Sleep Disorder"]], axis=1)
  return data_normalized

data = normalize_features(data)

#Compiles list of features to send to model and a label
def feed_to_model(delete_features):
  features = {name:np.array(value) for name, value in data.items()}
  label = np.array(features.pop("Has Sleep Disorder"))
  for feature in delete_features:
    del features[feature]
  return features, label

#List of features the model shouldn't use
delete_features = ["Person ID", "Sleep Disorder", "Occupation", "Gender", "Blood Pressure", "BMI Category"]
features, label = feed_to_model(delete_features)

#Creates dictionary assigning each feature to an input tensor
inputs = {}
for feature in features:
  inputs[feature] = tf.keras.Input(shape=(1,),dtype="float64")

#Concatenates all inputs to form one input layer
concatenated_inputs = tf.keras.layers.Concatenate()(inputs.values())
#Defines shape and activation function of the output layer
dense = layers.Dense(units=1, input_shape=(1,), activation=tf.sigmoid)
output = dense(concatenated_inputs)

#Defines the metrics we want to calculate while training
metrics=[tf.keras.metrics.BinaryAccuracy(threshold=classification_threshold),
         tf.keras.metrics.Precision(thresholds=classification_threshold),
         tf.keras.metrics.Recall(thresholds=classification_threshold)]

#Creates the model
model = tf.keras.Model(inputs=inputs, outputs=output)
model.compile(optimizer=tf.keras.optimizers.experimental.RMSprop(learning_rate=learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=metrics)

#Trains the model
history = model.fit(x=features, y=label, batch_size=batch_size, epochs=epochs, validation_split=0.2)

#Overall results from the model:
#Everytime the model is trained, it gets a slightly different result due to the nature of ML
#but here are averages after training the model 10 different times (rounded to 4 spots)
#Training Loss: 0.2477
#Training Accuracy: 92.01%
#Validation Loss: 0.3518
#Validation Accuracy: 89.87%

<ipython-input-10-43df098c9bac>:42: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data_normalized = (data - data.mean()) / data.std()
<ipython-input-10-43df098c9bac>:42: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data_normalized = (data - data.mean()) / data.std()


Epoch 1/100
25/25 [==============================] - 1s 23ms/step - loss: 0.7171 - binary_accuracy: 0.6254 - precision_9: 0.5882 - recall_9: 0.0870 - val_loss: 0.5400 - val_binary_accuracy: 0.6267 - val_precision_9: 0.9286 - val_recall_9: 0.3250
Epoch 2/100
25/25 [==============================] - 0s 7ms/step - loss: 0.3981 - binary_accuracy: 0.7358 - precision_9: 0.8333 - recall_9: 0.3913 - val_loss: 0.4497 - val_binary_accuracy: 0.6800 - val_precision_9: 0.9000 - val_recall_9: 0.4500
Epoch 3/100
25/25 [==============================] - 0s 6ms/step - loss: 0.3046 - binary_accuracy: 0.7759 - precision_9: 0.8529 - recall_9: 0.5043 - val_loss: 0.4337 - val_binary_accuracy: 0.7733 - val_precision_9: 0.8966 - val_recall_9: 0.6500
Epoch 4/100
25/25 [==============================] - 0s 6ms/step - loss: 0.2688 - binary_accuracy: 0.8896 - precision_9: 0.8942 - recall_9: 0.8087 - val_loss: 0.4356 - val_binary_accuracy: 0.8667 - val_precision_9: 0.9167 - val_recall_9: 0.8250
Epoch 5/100
25/25 [